In [1]:
import urllib.request
import requests
import time
import sys
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import pandas as pd
import datetime
import re
now = datetime.datetime.now()
import lxml

In [2]:
#datos = urllib.request.urlopen('https://www.forbes.com/real-time-billionaires').read().decode()
#url='https://www.forbes.com/real-time-billionaires'
# Pagina donde se empezó: https://forbes400.herokuapp.com/
# Pagina para API https://www.quandl.com/
url='https://forbes400.herokuapp.com/api/forbes400/getAllBillionaires'

In [3]:
response = requests.get(url)
html_soup = BeautifulSoup(response.text, "html.parser")

In [4]:
results = response.json()

In [5]:
tabla1 = json_normalize(results)

In [6]:
def displaydf(dataframe, cols = None, rows = 20):
    with pd.option_context("display.max_columns", cols):
        with pd.option_context("display.max_rows", rows):
            display(dataframe)
    return True

In [7]:
#displaydf(flattened_data)

In [8]:
tabla1=tabla1[['position','personName','city','countryOfCitizenship','gender','financialAssets','birthDate']]
#tabla1.birthDate = tabla1.birthDate.fillna(0)
#tabla1.birthDate = tabla1.birthDate.astype(int)

tabla1.loc[tabla1.birthDate.isnull(),'birthDate'] = 0


In [9]:
tabla1.loc[:,'birthDate'] = tabla1.birthDate.astype(int)

In [10]:
def title_COLUMN(df, column):
    df[column] = df[column].str.title()
    return df[column]
title_COLUMN(tabla1, 'personName')

0                     Jeff Bezos
1       Bernard Arnault & Family
2                     Bill Gates
3                 Warren Buffett
4                 Amancio Ortega
                  ...           
2180                 Morris Kahn
2181               Sachin Bansal
2182           Tatyana Bakalchuk
2183    Zhang Zhongneng & Family
2184             Miguel Mckelvey
Name: personName, Length: 2185, dtype: object

In [11]:
from datetime import datetime, timedelta

def birthDate_f(the_row):
    if the_row>0:
        date = datetime(1970, 1, 1) + timedelta(seconds=the_row/1000)
        return date.strftime('%Y')
    elif the_row<0:
        date = datetime(1970, 1, 1) - timedelta(seconds=-the_row/1000)
        return date.strftime('%Y')
    else:
        return 0
    
def Age_f(the_row):
    the_row = int(the_row) #añadido!!!!
    if the_row >= 1000:
        return now.year - the_row
    else:
        return the_row

def Gender_f(the_row):
    if the_row == "M":
        return "Male"
    elif the_row == "F":
        return "Female"
    else:
        return the_row

    
#tabla1['birthDate'] = tabla1.apply(lambda row: birthdate_f(row.birthDate), axis=1)
#tabla1['birthDate'] = tabla1.apply(lambda row: age_f(row.birthDate), axis=1)
#tabla1['gender'] = tabla1.apply(lambda row: gender_f(row.gender), axis=1)
tabla1.loc[:,'birthDate'] = tabla1.birthDate.apply(birthDate_f)
tabla1.loc[:,'birthDate'] = tabla1.birthDate.apply(Age_f)
tabla1.loc[:,'gender'] = tabla1.gender.apply(Gender_f)


In [12]:
tabla1

,position,personName,city,countryOfCitizenship,gender,financialAssets,birthDate
0,1,Jeff Bezos,Seattle,United States,Male,"[{'exchange': 'NASDAQ', 'ticker': 'AMZN-US', '...",56
1,2,Bernard Arnault & Family,Paris,France,Male,"[{'exchange': 'EURONEXT PARIS', 'ticker': 'CA-...",71
2,3,Bill Gates,Medina,United States,Male,"[{'exchange': 'NASDAQ', 'ticker': 'ACGL-US', '...",65
3,4,Warren Buffett,Omaha,United States,Male,"[{'exchange': 'NYSE', 'ticker': 'BRK.A-US', 'c...",90
4,5,Amancio Ortega,La Coruna,Spain,Male,"[{'exchange': 'MADRID', 'ticker': 'ITX-ES', 'c...",84
...,...,...,...,...,...,...,...
2180,2181,Morris Kahn,Beit Yanay,Israel,Male,NaN,90
2181,2182,Sachin Bansal,Bangalore,India,Male,NaN,39
2182,2183,Tatyana Bakalchuk,Moscow,Russia,Female,NaN,45
2183,2184,Zhang Zhongneng & Family,Shenzhen,China,Male,"[{'exchange': 'SHANGHAI', 'ticker': '600673-CN...",57


In [13]:
#tabla1.rename(columns = {'personName':'Name','position':'Position'}, inplace = True)

tabla1.rename(index={e: e+1 for e in range(len(tabla1))}, 
                columns={'personName':'Name','position':'Position'}, 
                inplace=True)

Recuperamos la tabla 2 del proyecto

In [14]:
tabla2 = pd.read_csv('../data/processed/ForbesBillionaires2018.csv', sep=',', index_col=0)

Vamos a extraer el codigo de la imagen para cruzar de la tabla 2

In [15]:
def Image_f(the_row):
    if the_row.startswith('https://') == True:
        the_row = str(re.findall('serve/[0-9a-z]+',the_row))
        return the_row[8:-2]
    else:
        return the_row

In [16]:
#tabla2['Image2'] = tabla2.apply(lambda row: Image_f(row.Image), axis=1)

El cruce entre tabla 1 y tabla 2

In [17]:
from functools import reduce
results = [tabla1, tabla2]
results1 = reduce(lambda left,right: pd.merge(left,right,on='Name', how='right'), results)

In [18]:
results1.birthDate = results1.birthDate.fillna(0)
results1.birthDate = results1.birthDate.astype(int)


In [19]:
def Age_none_f(age, birthDate):
    if age == 0:
        return (birthDate-1)
    else:
        return age
def Gender_none_f(Gender, gender):
    if Gender in (None, '','None'):
        return gender
    else:
        return Gender
def Country_none_f(Country, countryOfCitizenship):
    if Country in (None, '','None'):
        return countryOfCitizenship
    elif Country == 'USA':
        return 'United States'
    else:
        return Country

In [20]:
results1['Age'] = results1.apply(lambda row: Age_none_f(row.Age, row.birthDate), axis=1)
results1['Gender'] = results1.apply(lambda row: Gender_none_f(row.Gender, row.gender), axis=1)
results1['Country'] = results1.apply(lambda row: Country_none_f(row.Country, row.countryOfCitizenship), axis=1)

In [21]:
results1 = results1[['Position_y', 'Name', 'LastName', 'Age', 'Gender', 'Country', 'city', 'Worth', 'Image','Company','Sector','financialAssets']]

In [22]:
import requests
import lxml
import html5lib

from bs4 import BeautifulSoup

In [23]:
url = 'https://stats.areppim.com/listes/list_billionairesx18xwor.htm'
#url='https://data.opendatasoft.com/explore/embed/dataset/the-billionaire-characteristics-database@public-us/table/'
#url = 'https://www.reuters.com/article/us-shazam-m-a-apple-eu/eu-clears-apples-purchase-of-shazam-idUSKCN1LM1TZ'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')

In [24]:
table = soup.find_all('table',{'class':'stats'})[0]

In [25]:
#rows = table.find_all('tr')
#rows_parsed = [row for row in rows]
#rows_parsed

data = []
for i in table.findAll('tr'):
    data.append([j.text.strip() for j in i.findAll('td')])

import pandas as pd
tabla3 = pd.DataFrame(data[2:-3], columns=data[1])

In [26]:
title_COLUMN(tabla3, 'Name')

0            Jeff Bezos
1            Bill Gates
2        Warren Buffett
3       Bernard Arnault
4       Mark Zuckerberg
             ...       
2203     Zhao Xiaoqiang
2204    Zhou Liangzhang
2205       Zhu Xingming
2206           Zhuo Jun
2207        Olgun Zorlu
Name: Name, Length: 2208, dtype: object

In [27]:
from functools import reduce
results = [results1, tabla3]
results2 = reduce(lambda left,right: pd.merge(left,right,on='Name', how='left'), results)

In [28]:
def Country_none2_f(Country, Citizenship):
    if pd.isnull(Country) == True:
        return Citizenship
    elif Country == 'USA':
        return 'United States'
    else:
        return Country

In [29]:
results2['Country'] = results2.apply(lambda row: Country_none2_f(row.Country, row.Citizenship), axis=1)

In [30]:
results2 = results2[['Position_y', 'Name', 'LastName', 'Age_x', 'Gender', 'Country', 'city', 'Worth', 'Image','Company','Sector','financialAssets']]

In [31]:
results2.rename(columns = {'Age_x':'Age','Position_y':'Position','city':'City','financialAssets':'FinancialAssets'}, inplace = True)

In [32]:
results2 = results2.sort_values(by='Position', ascending=True)

In [33]:
results2 = results2.loc[results2.astype(str).drop_duplicates().index]

In [34]:
#results2.loc[results2['Name'] == 'Robert Miller']

In [35]:
#results2.loc[results2['Name'] == 'Li Li']

In [36]:
results2 = results2.drop(results2[(results2.Name == 'Li Li') & (results2.City == 'Shenzhen')].index)
results2 = results2.drop(results2[(results2.Name == 'Robert Miller') & (results2.City == 'Montreal') & (results2.Worth == 4.5)].index)
results2 = results2.drop(results2[(results2.Name == 'Robert Miller') & (results2.City == 'Hong Kong') & (results2.Worth == 2.6)].index)

In [37]:
results2 = results2.reset_index(drop=True)

In [38]:
results2.to_csv('../data/processed/ForbesBillionaires2018_2.csv', index=False)